# HUGO Scoring metrics:

In this challenge the goal is to create a model which can achieve 85% accuracy upto 2 hours earlier on given multifeatured dataset.

The essence of the task is similar to timeseries forecasting where using different algorithms, one can calculate the value of target variable in the future. The data can have temporal relationship among their different features. A successful solver should be able to analyze and leverage these detailed features in order to achieve the target accuracy.

Accuracy, this term is a very subjective term unless it is quantitatively measured. The goal of this analysis is to propose a framework which could objectively measure the accuracy of the proposed solvers and score them accordingly.

In the ideation challenge it was observed that there are 6 KPIs which needed to be forecasted. So the proposed scoring framework would assume 6 KPI scoring. However, the proposed framework is very loosely coupled (will be explained later), and can be used for any number of KPIs with very minimal tweaking.

In [24]:
## Load the necessary library
import numpy as np
import pandas as pd
import sys
import os

In [2]:
np.random.seed(2020)

In [3]:
"""
Let's assume solvers are challenged to predict next 30 days, each hour KPIs values
And we have the ground truth values for 30*24=720 points for those 6 KPIS.
The below snippet randomly (uniform and normal) generate ground truth value for 6 KPIs
"""
KPI1_GT = np.random.uniform(size=30*24,low=0, high=100)
KPI2_GT = np.random.normal(size=30*24, loc=5.0, scale=1.2)
KPI3_GT = np.random.uniform(size=30*24, low=0, high=12)
KPI4_GT = np.random.normal(size=30*24, loc=3.8, scale=0.24)
KPI5_GT = np.random.uniform(size=30*24, low=1.5, high=45.78)
KPI6_GT = np.random.normal(size=30*24, loc=120, scale=50)

In [4]:
KPI1_GT.shape

(720,)

In [10]:
pd.DataFrame\
(np.column_stack([KPI1_GT,
                 KPI2_GT,
                 KPI3_GT,
                 KPI4_GT,
                 KPI5_GT,
                 KPI6_GT
                ]),
 columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
).to_csv("ground_truth_file.csv", header=True, index=False)

In [11]:
"""
Let us assume in the submission phase 7 participants submitted
"""
participant_kpi_prediction = {}
participant_kpi_prediction["participant_1"] = np.column_stack([np.random.uniform(size=30*24,low=4, high=120),
                                                               np.random.normal(size=30*24, loc=2.0, scale=1.2),
                                                               np.random.uniform(size=30*24, low=5, high=10),
                                                               np.random.normal(size=30*24, loc=3.8, scale=0.24),
                                                               np.random.uniform(size=30*24, low=5.5, high=35.78),
                                                               np.random.normal(size=30*24, loc=100, scale=50)
                                                               ])

In [12]:
"""
Here rows are 30 days * 24 hours and columns are different KPIs
"""
participant_kpi_prediction["participant_1"].shape


(720, 6)

In [13]:
## Randomly populate other 6 particpants prediction
participant_kpi_prediction["participant_2"] = np.column_stack([np.random.uniform(size=30*24,low=4, high=120),
                                                               np.random.normal(size=30*24, loc=2.0, scale=1.2),
                                                               np.random.uniform(size=30*24, low=5, high=10),
                                                               np.random.normal(size=30*24, loc=3.8, scale=0.24),
                                                               np.random.uniform(size=30*24, low=5.5, high=35.78),
                                                               np.random.normal(size=30*24, loc=100, scale=50)
                                                               ])
participant_kpi_prediction["participant_3"] = np.column_stack([np.random.uniform(size=30*24,low=4, high=120),
                                                               np.random.normal(size=30*24, loc=2.0, scale=1.2),
                                                               np.random.uniform(size=30*24, low=5, high=10),
                                                               np.random.normal(size=30*24, loc=3.8, scale=0.24),
                                                               np.random.uniform(size=30*24, low=5.5, high=35.78),
                                                               np.random.normal(size=30*24, loc=100, scale=50)
                                                               ])
participant_kpi_prediction["participant_4"] = np.column_stack([np.random.uniform(size=30*24,low=4, high=120),
                                                               np.random.normal(size=30*24, loc=2.0, scale=1.2),
                                                               np.random.uniform(size=30*24, low=5, high=10),
                                                               np.random.normal(size=30*24, loc=3.8, scale=0.24),
                                                               np.random.uniform(size=30*24, low=5.5, high=35.78),
                                                               np.random.normal(size=30*24, loc=100, scale=50)
                                                               ])
participant_kpi_prediction["participant_5"] = np.column_stack([np.random.uniform(size=30*24,low=4, high=120),
                                                               np.random.normal(size=30*24, loc=2.0, scale=1.2),
                                                               np.random.uniform(size=30*24, low=5, high=10),
                                                               np.random.normal(size=30*24, loc=3.8, scale=0.24),
                                                               np.random.uniform(size=30*24, low=5.5, high=35.78),
                                                               np.random.normal(size=30*24, loc=100, scale=50)
                                                               ])
participant_kpi_prediction["participant_6"] = np.column_stack([np.random.uniform(size=30*24,low=4, high=120),
                                                               np.random.normal(size=30*24, loc=2.0, scale=1.2),
                                                               np.random.uniform(size=30*24, low=5, high=10),
                                                               np.random.normal(size=30*24, loc=3.8, scale=0.24),
                                                               np.random.uniform(size=30*24, low=5.5, high=35.78),
                                                               np.random.normal(size=30*24, loc=100, scale=50)
                                                               ])
participant_kpi_prediction["participant_7"] = np.column_stack([np.random.uniform(size=30*24,low=4, high=120),
                                                               np.random.normal(size=30*24, loc=2.0, scale=1.2),
                                                               np.random.uniform(size=30*24, low=5, high=10),
                                                               np.random.normal(size=30*24, loc=3.8, scale=0.24),
                                                               np.random.uniform(size=30*24, low=5.5, high=35.78),
                                                               np.random.normal(size=30*24, loc=100, scale=50)
                                                               ])

In [16]:
pd.DataFrame(participant_kpi_prediction["participant_1"],
             columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
            ).to_csv("participants_forecast/participant_1.csv", header=True, index=False)

In [17]:
pd.DataFrame(participant_kpi_prediction["participant_2"],
             columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
            ).to_csv("participants_forecast/participant_2.csv", header=True, index=False)

In [18]:
pd.DataFrame(participant_kpi_prediction["participant_3"],
             columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
            ).to_csv("participants_forecast/participant_3.csv", header=True, index=False)

In [19]:
pd.DataFrame(participant_kpi_prediction["participant_4"],
             columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
            ).to_csv("participants_forecast/participant_4.csv", header=True, index=False)

In [20]:
pd.DataFrame(participant_kpi_prediction["participant_5"],
             columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
            ).to_csv("participants_forecast/participant_5.csv", header=True, index=False)

In [21]:
pd.DataFrame(participant_kpi_prediction["participant_6"],
             columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
            ).to_csv("participants_forecast/participant_6.csv", header=True, index=False)

In [22]:
pd.DataFrame(participant_kpi_prediction["participant_7"],
             columns=["KPI1","KPI2","KPI3","KPI4","KPI5","KPI6"]
            ).to_csv("participants_forecast/participant_7.csv", header=True, index=False)

To identify the accuracy, the atomic proposed error metric is [sMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error). This means the forecasted and actual ground truth values of the KPIs will be compared using sMAPE. We will use the variant which gives value between 0-100% error.

Idea is that, if we get an error percentage, then the accuracy value would be 100-error percentage. Also sMAPE is prone to be infinite in case actual and forecast both are zero, that is why we have added a very small number 1e-10 with each of the actual and forecasted values.

In [23]:
## SMAPE definition
"""
Notice 0.5 factor on denominator is ignored to get the error between 0-100
"""
def smape(A, F, z=1e-10):
    return 100/len(A) * np.sum(np.abs(F - A) / (np.abs(A) + np.abs(F) + z))


In [0]:
"""
Using the above formula one can calculate SMAPE value and subsequently the accuracy value as 1-sMAPE
Goal is to maximize this 1 - sMAPE for individual participant
"""
## Scoring for KPI1
kpi1_scoring = []
for participant in participant_kpi_prediction.keys():
    forecast = participant_kpi_prediction[participant][:,0]
    actual = KPI1_GT
    kpi1_scoring.append(100 - smape(actual, forecast))

In [10]:
kpi1_scoring

[61.89984645383983,
 61.92205901789102,
 63.22198864018862,
 62.3248663419783,
 62.57801586071289,
 61.73841649797492,
 61.51133896249746]

In [0]:
"""
Using the above formula one can calculate SMAPE value and subsequently the accuracy value as 1-sMAPE
Goal is to maximize this 100 - sMAPE for individual participant
"""
## Scoring for KPI2
kpi2_scoring = []
for participant in participant_kpi_prediction.keys():
    forecast = participant_kpi_prediction[participant][:,1]
    actual = KPI2_GT
    kpi2_scoring.append(100 - smape(actual, forecast))

In [12]:
kpi2_scoring

[53.85787764763254,
 56.57845871636669,
 54.67481177270937,
 54.07044137660395,
 55.16921489272388,
 54.54284278094327,
 53.62578330755656]

In [0]:
"""
Using the above formula one can calculate SMAPE value and subsequently the accuracy value as 1-sMAPE
Goal is to maximize this 100 - sMAPE for individual participant
"""
## Scoring for KPI3
kpi3_scoring = []
for participant in participant_kpi_prediction.keys():
    forecast = participant_kpi_prediction[participant][:,2]
    actual = KPI3_GT
    kpi3_scoring.append(100 - smape(actual, forecast))

In [14]:
kpi3_scoring

[68.75912587176867,
 68.33281715262777,
 68.46195082049732,
 68.32902267144608,
 68.46307390006625,
 68.21501105665861,
 68.53778333388816]

In [0]:
"""
Using the above formula one can calculate SMAPE value and subsequently the accuracy value as 1-sMAPE
Goal is to maximize this 100 - sMAPE for individual participant
"""
## Scoring for KPI4
kpi4_scoring = []
for participant in participant_kpi_prediction.keys():
    forecast = participant_kpi_prediction[participant][:,3]
    actual = KPI4_GT
    kpi4_scoring.append(100 - smape(actual, forecast))

In [16]:
kpi4_scoring

[96.404090913865,
 96.50976905909349,
 96.35566375929601,
 96.41654887477706,
 96.31911695893974,
 96.45324933128128,
 96.42733927980527]

In [0]:
"""
Using the above formula one can calculate SMAPE value and subsequently the accuracy value as 1-sMAPE
Goal is to maximize this 100 - sMAPE for individual participant
"""
## Scoring for KPI5
kpi5_scoring = []
for participant in participant_kpi_prediction.keys():
    forecast = participant_kpi_prediction[participant][:,4]
    actual = KPI5_GT
    kpi5_scoring.append(100 - smape(actual, forecast))

In [18]:
kpi5_scoring

[69.15969566777977,
 67.75940914981767,
 68.09392409402777,
 67.99220614794825,
 68.77039287358112,
 68.39548511920198,
 67.70253584252525]

In [0]:
"""
Using the above formula one can calculate SMAPE value and subsequently the accuracy value as 1-sMAPE
Goal is to maximize this 100 - sMAPE for individual participant
"""
## Scoring for KPI6
kpi6_scoring = []
for participant in participant_kpi_prediction.keys():
    forecast = participant_kpi_prediction[participant][:,5]
    actual = KPI6_GT
    kpi6_scoring.append(100 - smape(actual, forecast))

In [20]:
kpi6_scoring

[70.9130833578896,
 69.92501774002791,
 69.17324546775845,
 70.45711507421956,
 70.93368848849855,
 70.80460964609071,
 71.17031351958641]

After getting the score for each participants for each of the KPI, we should normalize the score, in order to rank participants in the competition.

In [0]:
## Normalize KPI scores for each participants (divide each participant's score by max score of that KPI)
normalized_kpi1_score = np.array(kpi1_scoring)/np.max(np.array(kpi1_scoring))
normalized_kpi2_score = np.array(kpi2_scoring)/np.max(np.array(kpi2_scoring))
normalized_kpi3_score = np.array(kpi3_scoring)/np.max(np.array(kpi3_scoring))
normalized_kpi4_score = np.array(kpi4_scoring)/np.max(np.array(kpi4_scoring))
normalized_kpi5_score = np.array(kpi5_scoring)/np.max(np.array(kpi5_scoring))
normalized_kpi6_score = np.array(kpi6_scoring)/np.max(np.array(kpi6_scoring))

In [22]:
normalized_kpi1_score

array([0.9790873 , 0.97943865, 1.        , 0.98580996, 0.9898141 ,
       0.97653392, 0.97294217])

In [0]:
## Combine the scores
normalized_combined_kpi_score = np.column_stack([normalized_kpi1_score,
                                                 normalized_kpi2_score,
                                                 normalized_kpi3_score,
                                                 normalized_kpi4_score,
                                                 normalized_kpi5_score,
                                                 normalized_kpi6_score
                                                 ])

In [24]:
"""
Rows denotes the participants and columns denotes the KPIs
"""
normalized_combined_kpi_score

array([[0.9790873 , 0.9519149 , 1.        , 0.998905  , 1.        ,
        0.99638571],
       [0.97943865, 1.        , 0.99379997, 1.        , 0.97975285,
        0.9825026 ],
       [1.        , 0.96635386, 0.99567803, 0.99840322, 0.9845897 ,
        0.97193959],
       [0.98580996, 0.95567187, 0.99374478, 0.99903409, 0.98311893,
        0.98997899],
       [0.9898141 , 0.97509222, 0.99569436, 0.99802453, 0.99437096,
        0.99667523],
       [0.97653392, 0.96402136, 0.99208665, 0.99941436, 0.98895006,
        0.99486157],
       [0.97294217, 0.94781273, 0.9967809 , 0.99914589, 0.97893051,
        1.        ]])

In [0]:
summed_up_kpi_score_for_participant = np.sum(normalized_combined_kpi_score, axis=1)

In [26]:
## Summed up score for each participants - maximum score possible is 6 (1 for each KPI)
"""
The proposed scoring framework is loosely coupled because 
one can calculate any number of KPIS and participants using this framework
"""
summed_up_kpi_score_for_participant

array([5.92629291, 5.93549406, 5.91696439, 5.90735862, 5.9496714 ,
       5.91586792, 5.8956122 ])

In [27]:
## Ranking of participants in ascending order
"""
as it is 0 based index

1st - participant 5
2nd - participant 2
3rd - participant 1
4th - participant 3
5th - participant 6
6th - participant 4
7th - participant 7
"""
np.argsort(summed_up_kpi_score_for_participant)

array([6, 3, 5, 2, 0, 1, 4])

In case there is a tie in the above scoring metric then one can judge the performance of the model based on number of hours prior the model can forecast the target value. In the ideation phase, the top ideation have tried and proved that the performance could be same even if the prediction is done 24 hours earlier. Thus there is an opportunity to enhance this aspect of forecasting.

This will help business to get insight much earlier and thus gaining competitive advantage.

As an example : If participant 1 and 2 both achieves score 5.2 then if participant 1 achieves the score by predicting 4 hours earlier and participant 2 achieves the score by predicting 3 hours earlier then participant 1 sould be the winner.

In the above example, let's assume task is to predict the KPI value at 11AM. Partipant 1 and 2 both predict with similar accuracy (explained above) but participant 1 was able to predict by using data till 7AM and participant 2 was able to predict by using data till 8AM. Both of the model took 30 mins to predict. Then we can say that the model of participant 1 gave 1 hour advantage over the model of participant 2.

No model should use the data later than 2 hours prior (e.g. for 11AM prediction max data range could be same day 9AM).

If the tie still remains, then participant's model explainability should be considered for subjectively break the tie. Tiebreaker should not be on the submission timing.

In [55]:
"""
Create a helper function to assist reviewer to score for all the participants
"""
def calculate_combined_score(ground_truth_file, forecast_directory):
    ground_truth_df = pd.read_csv(ground_truth_file)
    participant_detailed_forecast = []
    file_list = os.listdir(forecast_directory)
    for file in file_list:
        file_id = str(file).split(".")[0]
        file_df = pd.read_csv(os.path.join(forecast_directory,file))
        file_df["id"] = file_id
        participant_detailed_forecast.append(file_df)
        
    participant_detailed_forecast = \
    pd.concat(participant_detailed_forecast, sort=True)
    
    kpi1_scoring = []
    kpi2_scoring = []
    kpi3_scoring = []
    kpi4_scoring = []
    kpi5_scoring = []
    kpi6_scoring = []
    
    participants_id = participant_detailed_forecast["id"].unique()
    
    # KPI1 scoring
    for participant in participants_id:
        forecast = \
        participant_detailed_forecast[participant_detailed_forecast["id"]==participant].\
        iloc[:,0].values
        actual = ground_truth_df.iloc[:,0].values
        kpi1_scoring.append(100 - smape(actual, forecast))
        
    # KPI2 scoring
    for participant in participants_id:
        forecast = \
        participant_detailed_forecast[participant_detailed_forecast["id"]==participant].\
        iloc[:,1].values
        actual = ground_truth_df.iloc[:,1].values
        kpi2_scoring.append(100 - smape(actual, forecast))
        
    # KPI3 scoring
    for participant in participants_id:
        forecast = \
        participant_detailed_forecast[participant_detailed_forecast["id"]==participant].\
        iloc[:,2].values
        actual = ground_truth_df.iloc[:,2].values
        kpi3_scoring.append(100 - smape(actual, forecast))
        
    # KPI4 scoring
    for participant in participants_id:
        forecast = \
        participant_detailed_forecast[participant_detailed_forecast["id"]==participant].\
        iloc[:,3].values
        actual = ground_truth_df.iloc[:,3].values
        kpi4_scoring.append(100 - smape(actual, forecast))
        
    # KPI5 scoring
    for participant in participants_id:
        forecast = \
        participant_detailed_forecast[participant_detailed_forecast["id"]==participant].\
        iloc[:,4].values
        actual = ground_truth_df.iloc[:,4].values
        kpi5_scoring.append(100 - smape(actual, forecast))
        
        
    # KPI6 scoring
    for participant in participants_id:
        forecast = \
        participant_detailed_forecast[participant_detailed_forecast["id"]==participant].\
        iloc[:,5].values
        actual = ground_truth_df.iloc[:,5].values
        kpi6_scoring.append(100 - smape(actual, forecast))
        
    
        
    ## Normalize KPI scores for each participants (divide each participant's score by max score of that KPI)
    normalized_kpi1_score = np.array(kpi1_scoring)/np.max(np.array(kpi1_scoring))
    normalized_kpi2_score = np.array(kpi2_scoring)/np.max(np.array(kpi2_scoring))
    normalized_kpi3_score = np.array(kpi3_scoring)/np.max(np.array(kpi3_scoring))
    normalized_kpi4_score = np.array(kpi4_scoring)/np.max(np.array(kpi4_scoring))
    normalized_kpi5_score = np.array(kpi5_scoring)/np.max(np.array(kpi5_scoring))
    normalized_kpi6_score = np.array(kpi6_scoring)/np.max(np.array(kpi6_scoring))
    
    
    ## Combine the scores
    normalized_combined_kpi_score = np.column_stack([normalized_kpi1_score,
                                                     normalized_kpi2_score,
                                                     normalized_kpi3_score,
                                                     normalized_kpi4_score,
                                                     normalized_kpi5_score,
                                                     normalized_kpi6_score
                                                     ])
    
    
    ## Summing up
    summed_up_kpi_score_for_participant = np.sum(normalized_combined_kpi_score, axis=1)
    
    score_df = \
    pd.DataFrame(summed_up_kpi_score_for_participant,columns=["normalized_score"])
    
    score_df["participant_id"] = participants_id
    
    return score_df.sort_values(by="normalized_score", ascending=False)

In [56]:
calculate_combined_score("ground_truth_file.csv","participants_forecast")

normalized_score participant_id
3          5.949671  participant_5
5          5.935494  participant_2
4          5.926293  participant_1
6          5.916964  participant_3
1          5.915868  participant_6
2          5.907359  participant_4
0          5.895612  participant_7